In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/df_01.csv')

In [3]:
lista = list(df['CPF_CNPJ_Des'])
lista

[5291847964,
 5668336972,
 5668336972,
 5269738970,
 3970921996,
 35442751987,
 4938853906,
 61652717900,
 5451662924,
 5451662924,
 5451662924,
 5451662924,
 44767927900,
 7213760963,
 1881864928,
 663340985,
 8210284991,
 6070065875,
 5947312913,
 3594546900,
 3594546900,
 969279922,
 2799445942,
 2799445942,
 2799445942,
 30741823934,
 65850700978,
 5365484983,
 671829920,
 55521495991,
 55521495991,
 9185454982,
 50046330968,
 78093945920,
 31691510963,
 62639897934,
 2291702939,
 3068239931,
 25318578934,
 25318578934,
 1989363970,
 1989363970,
 40358160944,
 40358160944,
 70051321904,
 80379419904,
 6321160997,
 5956762942,
 48254452920,
 53004760991,
 5211577000166,
 6581044970,
 8442230939,
 14255645850,
 934180946,
 934180946,
 4323422903,
 4323422903,
 27844722991,
 27844722991,
 27844722991,
 30742897915,
 34055223900,
 34055223900,
 47092254949,
 48050814904,
 4607452980,
 96514817904,
 9451008845,
 8710708987,
 2057962903,
 2057962903,
 2057962903,
 3607819939,
 3607819939

In [4]:
import pandas as pd
import networkx as nx
import numpy as np
from networkx.classes.function import path_weight
from tqdm import tqdm



In [5]:

def get_concessions(list_nodes: list,emp_type: dict)-> int: 
  # As concessões são todas emps marcadas como MANEJO,1
  # (fonte legal e extratores de madeira)

  count = 0
  for node in list_nodes:
    if emp_type[node] == 'MANEJO':
      count+=1

  return count

def get_sink_nodes(graph: nx.graph, emp_type: dict)-> dict:
  # If node is marked as FINAL, he is a sink
  # It is the final destination of the timber chain

  nodes = {}
  for node in graph.nodes():
    if emp_type[node] == 'FINAL':
      nodes[node] = 1
      continue

    # we consider sink nodes as final nodes or nodes that only transports to other final nodes
    not_sink = False
    for edge in graph.edges(node):
      if emp_type[edge[1]] != 'FINAL':
        not_sink=True

    if not not_sink:
      nodes[node] = 1

  return nodes

def get_timberflow(graph: nx.graph,emp_type: dict) -> None:
  
  sink_nodes = get_sink_nodes(graph, emp_type)

  total_in = 0
  total_out = 0

  # For all edges in the graph
  # 1. sum the volume of input (from MANEJO types)
  # 2. sum the volume of output (from FINAL types)
  for edge in graph.edges():
    if emp_type[edge[0]] == 'MANEJO':
      total_in += path_weight(graph, [edge[0],edge[1]], weight='Volume')
    elif edge[1] in sink_nodes:
      total_out += path_weight(graph, [edge[0],edge[1]], weight='Volume')

  print(f'Inflow Vol(m3): {total_in} \nOut Vol(m3): {total_out} \nProportion: {total_out/total_in}')

In [6]:
def convert_id_to_str(df: pd.DataFrame )-> pd.DataFrame:
    """Converte  as colunas de CPF_CNPJ_Rem e CPF_CNPJ_Des
    para str

    Args:
        df (pd.DataFrame): _description_

    Returns:
        pd.DataFrame: _description_
    """
    df['CPF_CNPJ_Rem'] = df['CPF_CNPJ_Rem'].astype(str)
    df['CPF_CNPJ_Des'] = df['CPF_CNPJ_Des'].astype(str)
    
    return df

In [8]:

# transportes_junho = pd.read_csv('../data/df_06.csv')
transportes_julho = pd.read_csv('../data/df_07.csv')
transportes_agosto = pd.read_csv('../data/df_08.csv')
transportes_setembro = pd.read_csv('../data/df_09.csv')
transportes_outubro= pd.read_csv('../data/df_10.csv')
transportes_novembro = pd.read_csv('../data/df_11.csv')
transportes_dezembro = pd.read_csv('../data/df_12.csv')

tranporte_segundo_semestre= pd.concat([transportes_julho, transportes_agosto, transportes_setembro, transportes_outubro, transportes_novembro, transportes_dezembro], ignore_index=True)

df_tran = tranporte_segundo_semestre[['CPF_CNPJ_Rem', 'TpRem', 'CPF_CNPJ_Des', 'TpDes', 'Volume']]
df_tran= df_tran.groupby(['CPF_CNPJ_Rem', 'TpRem', 'CPF_CNPJ_Des', 'TpDes'])['Volume'].sum().reset_index()
df_tran = convert_id_to_str(df_tran)


#  Empresas do tipo Patio q_ue transportam para empresas do tipo pátio
df_pto = df_tran[(df_tran['TpRem'] == 'PTO_IBAMA') & (df_tran['TpDes'] == 'PTO_IBAMA')].rename(columns={'CPF_CNPJ_Rem': 'CPF_CNPJ'})
df_pto = df_pto.groupby('CPF_CNPJ')['Volume'].sum().reset_index()

nodes = set(df_tran['CPF_CNPJ_Rem']).union(set(df_tran['CPF_CNPJ_Des']))
nodes_pto = set(df_pto['CPF_CNPJ'])

#  Criando grafo
G = nx.DiGraph()
G.add_nodes_from(nodes)


# Cria as arestas com base nas transações e com peso = volume
edges = []
for row in df_tran.iterrows():
    # Ignora laços
    if str(row[1]['CPF_CNPJ_Rem']) != str(row[1]['CPF_CNPJ_Des']):
        edges.append((str(row[1]['CPF_CNPJ_Rem']), str(row[1]['CPF_CNPJ_Des']), {'Volume': row[1]['Volume']}))
G.add_edges_from(edges)



emp_pto_degree = {}

for node in nodes_pto:
    emp_pto_degree[node]= G.degree(node)

graus = list(emp_pto_degree.values())


# print(f"Grau:    {graus}")
q1 = np.quantile(graus, 0.25)  # Primeiro quartil (25%)
q2 = np.quantile(graus, 0.5)   # Segundo quartil (50%, ou mediana)
q3 = np.quantile(graus, 0.75)  # Terceiro quartil (75%)
interquartil = q3-q1
limite_superior = q3 + 1.5*interquartil
print(f"Q1 (25%): {q1}")
print(f"Q2 (50% - Mediana): {q2}")
print(f"Q3 (75%): {q3}")
print(f"Limite superior: {limite_superior}")
print(f"Máximo: {max(graus)}")


pto_outliers = 0 

for emp in graus:
  if emp>= limite_superior:
    pto_outliers+=1

print(f"Numero de Empresas Patio outliers (importantes quanto grau): {pto_outliers} ")

print()
numero_original_de_componentes =  nx.number_weakly_connected_components(G)
print(f"Número total de componentes conexas: {numero_original_de_componentes}")


#  Adicionando  coluna de grau de um node
df_pto['Grau'] = df_pto['CPF_CNPJ'].apply(lambda x: G.degree(x))


#  Filtrando os outliers do tipo patio

df_pto_outliers = df_pto[df_pto['Grau']>= limite_superior]
df_pto_outliers.loc[:, 'is_bridge_linkage'] = False


# Analisando  o numero de componentes que 
#  são obtidas ao remover os vertices pto 

components = []

print()
print("Analisando componentes dos possíveis pontos de articulação")
print("...")
for node in list(df_pto_outliers['CPF_CNPJ']):
    SG = nx.subgraph_view(G, filter_node= lambda x: x != node)
    components.append(nx.number_weakly_connected_components(SG))
print("Analisado")

df_pto_outliers['Conected_Components']= components
df_pto_outliers.loc[df_pto_outliers['Conected_Components'] > numero_original_de_componentes, 'is_bridge_linkage'] = True

print()
qtd_de_pontes_de_articulacao =  df_pto_outliers['is_bridge_linkage'].sum()
print(f"Das empresas importantes, quantas são pontos de  de articulação: {qtd_de_pontes_de_articulacao}")

# CNPJ/CPFS das empresas importantes
importante_nodes = list(df_pto_outliers['CPF_CNPJ'])

#  Componentes fracamente conexas
weakly_connected_components = list(nx.weakly_connected_components(G))

# result = [
#     component
#     for component in weakly_connected_components
#     if len(set(component) & set(importante_nodes)) > 1
# ]

# num_components_with_important = len(result)



C:\Users\jaime\AppData\Local\Temp\ipykernel_10704\3596949724.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  transportes_julho = pd.read_csv('../data/df_07.csv')
C:\Users\jaime\AppData\Local\Temp\ipykernel_10704\3596949724.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  transportes_agosto = pd.read_csv('../data/df_08.csv')
C:\Users\jaime\AppData\Local\Temp\ipykernel_10704\3596949724.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  transportes_setembro = pd.read_csv('../data/df_09.csv')
C:\Users\jaime\AppData\Local\Temp\ipykernel_10704\3596949724.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  transportes_outubro= pd.read_csv('../data/df_10.csv')
C:\Users\jaime\AppData\Local\Temp\ipykernel_10704\3596949724.py:6: DtypeWarning: Columns (4) have mixed typ

Q1 (25%): 3.0
Q2 (50% - Mediana): 8.0
Q3 (75%): 22.0
Limite superior: 50.5
Máximo: 1105
Numero de Empresas Patio outliers (importantes quanto grau): 318 

Número total de componentes conexas: 818

Analisando componentes dos possíveis pontos de articulação
...


C:\Users\jaime\AppData\Local\Temp\ipykernel_10704\3596949724.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pto_outliers.loc[:, 'is_bridge_linkage'] = False


Analisado

Das empresas importantes, quantas são pontos de  de articulação: 318


C:\Users\jaime\AppData\Local\Temp\ipykernel_10704\3596949724.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pto_outliers['Conected_Components']= components


In [ ]:

# CNPJ/CPFS das empresas importantes
importante_nodes = list(df_pto_outliers['CPF_CNPJ'])

#  Componentes fracamente conexas
weakly_connected_components = list(nx.weakly_connected_components(G))


result = [
    set(component) & set(importante_nodes)
    for component in weakly_connected_components
    if len(set(component) & set(importante_nodes)) > 1
]

num_components_with_important = len(result)

print(f"Numero de componentes conexas com mais de uma empresa importante: {num_components_with_important}")

print(f"Conjuntos de nós importantes em componentes fracamente conexas:")
for comp in result:
    print(f"Numero de vertices importantes nessa componente: {len(comp)}")
    print("Vertices importantes nessa componente:")
    print(comp)
    print()

Numero de componentes conexas com mais de uma empresa importante: 2
Conjuntos de nós importantes em componentes fracamente conexas:
Numero de vertices importantes nessa componente: 311
Vertices importantes nessa componente:
{'58838921000108', '10310351000152', '6993439000149', '22488714000123', '3865037000170', '1589479000106', '55844229000366', '3281332000180', '7386707000127', '79109088000134', '10490954000183', '5168614000109', '10933639000183', '53314274000111', '7832434000105', '95817664000154', '62878319000171', '17931108000109', '2174193000122', '3265615000138', '9341951000107', '3407641000153', '4573040000183', '22153766000149', '43538362000143', '6337318000149', '73304982000189', '3846857000115', '58152711000161', '7120780000152', '26416168000176', '95808259000170', '15801072000105', '85491520000105', '67514877000171', '27588000177', '7708343000154', '21780629000171', '48508402000551', '9294655000194', '9427334000110', '5661374000171', '1148278000173', '76313394000108', '57580

In [14]:


# CNPJ/CPFS das empresas importantes
importante_nodes = list(df_pto_outliers['CPF_CNPJ'])


strongly_connected_components = list(nx.strongly_connected_components(G))

# Filtrar componentes com mais de um nó importante, mantendo apenas os nós importantes
result = [
    set(component) & set(importante_nodes)
    for component in strongly_connected_components
    if len(set(component) & set(importante_nodes)) > 1
]
print(f'Numero de componentes fortemente conexas com mais de uma empresa importante: {len(result)} ')
print(f"Conjuntos de nós importantes em componentes fortemente conexas:")

for comp in result:
    print(f"Numero de vertices importantes nessa componente: {len(comp)}")
    print("Vertices importantes nessa componente:")
    print(comp)
    print()

Numero de componentes fortemente conexas com mais de uma empresa importante: 2 
Conjuntos de nós importantes em componentes fortemente conexas:
Numero de vertices importantes nessa componente: 2
Vertices importantes nessa componente:
{'4115760000103', '4223482000108'}

Numero de vertices importantes nessa componente: 2
Vertices importantes nessa componente:
{'9094931000170', '59229278000188'}

